In [2]:
# main URL area, contain for each novel 
url = "https://www.xbiquge.so/book/49549/"

In [6]:
# import area
import requests
from bs4 import BeautifulSoup
import numpy as np

In [4]:
# Parse the html text and retreive all the chapters 
r = requests.get(url)
soup = BeautifulSoup(r.text)

# Since BiQuGe use cache in their website, we need to drop first 12 website
# which are the newest 12 chapters and have been included later
sub_url_list = soup.find(id='list').find_all('a')[12:]
sub_url_list = [sub_url.get("href") for sub_url in sub_url_list]
title = str(soup.find(id='list').find("dt")).split("》")[0].split("《")[1]

In [5]:
# By checking the corresponding website to retrieve content
def get_text_and_title(url, sub_url):
#     sub_url = sub_url_list[0]
    chapter_url = url + sub_url
    r = requests.get(chapter_url)
    soup = BeautifulSoup(r.text)
    sub_text = soup.find(id='content').text.split("\xa0\xa0\xa0\xa0")[1:]
    sub_title = soup.find("title").text.split("_")[0]
    return sub_text, sub_title

In [7]:
# the function used in multi-thread
def get_list_text(selected_list):
    # selected_list: list like, [[index of the chapter, url of the chapter], ...]
    ret = []
    for index, sub_url in selected_list:
        sub_text, sub_title = get_text_and_title(url, sub_url)
        ret.append([index, sub_title, sub_text])
    return ret

In [14]:
workers = 10
indexed_list = [[index, i] for index, i in enumerate(sub_url_list)]
n_workers = list(np.array_split(indexed_list, workers))
for i in n_workers:
    

321
321
321
321
321
321
320
320
320
320
